# Load the Libraries

In [1]:
!pip install auto-gptq
!pip -q install langchain tiktoken
!pip -q install chromadb
!pip -q install sentence_transformers
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 115.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the test document

In [4]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('/content/drive/MyDrive/kagglex/test.pdf')
documents = loader.load()

# Load the documents into vector database

In [5]:
len(documents)

10

In [6]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

20


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

# Download model from Hugging face
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [8]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
from langchain.vectorstores import Chroma
## Here is the nmew embeddings being used
embedding = hf_embed

vectordb = Chroma.from_documents(documents=docs,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [9]:
def get_similar_docs(question, similar_doc_count):
  return vectordb.similarity_search(question, k=similar_doc_count)

# Similarity search testing

In [10]:
for doc in get_similar_docs("question your doc?", 3):
  print(doc.page_content)

- 8 - 
  
 
 
Are you aware of any other doctor who holds a different professional opinion 
regarding the patient’ s mental capacity? If so, please provide details:  
 
 
No. 
 
 
 
 
 
  
PROGNOSIS  
 
In your opinion, is the patient likely to regain mental capacity?  
 
 Yes  No         Not Sure  
 
If “Yes” or “Not Sure”, please suggest when another assessment of the patient’s 
mental capacity should be carr ied out:  
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
In your opinion, would the patient understand if he/she were to be informed of 
this application?  
 Yes             No
room flat.) I asked Mr Tan what he wanted to do with the flat, since he was not 
staying there. He said that he wanted to rent it  out. I asked him whether he planned to 
rent o ut the whole flat. Mr Tan said yes (even though his elderly mother was still 
living there.)  I asked him how much rent he planned to charge,  Mr Tan said he did not 
know , and that he was not good at money matters .  
 
I a

# Create a retriever for collecting atleast 3 simialr documents

In [11]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Load the 7B llama 2 model

In [12]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_basename = "model"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

# Set the prompts

In [13]:
prompt = "Tell me about AI"
system_message = "You are a helpful, respectful and honest medical assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids,do_sample=True, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:
<s> [INST] <<SYS>>
You are a helpful, respectful and honest medical assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Tell me about AI [/INST]  Hello! I'm here to help you with any questions you may have. AI, or Artificial Intelligence, is a rapidly growing field that involves the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, and decision-making.
There are many different types of AI, including:
1. Narrow or weak AI: This type of AI is designed to perform

In [14]:
from langchain.prompts import PromptTemplate
template = """[INST] <<SYS>>
Use the following pieces of context to answer the medical question about the patient at the end. If you don't know the answer,\
just say that you don't know, don't try to make up an answer.
<</SYS>>
{context}
{question}
[/INST]
"""
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

# Build a pipeline

In [15]:
# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

print("* Pipeline:")
pipe = pipeline(
    "text-generation",
    do_sample=True,
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

* Pipeline:


In [16]:
from langchain.llms import HuggingFacePipeline
hf_pipe = HuggingFacePipeline(pipeline=pipe)

In [17]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=hf_pipe, chain_type="stuff", retriever=retriever,return_source_documents=True, chain_type_kwargs = {"prompt": prompt})

# Start chatting with the model

In [18]:
while True:
  query = input("\nEnter a query: ")
  if query == "exit":
      break
  # Get the answer from the chain
  res = qa(query)
  answer, docs = res["result"], res['source_documents']

  # Print the result
  print("\n\n> Question:")
  print(query)
  print("\n> Answer:")
  print(answer)
  print("----------------------------------SOURCE DOCUMENTS---------------------------")
  for document in docs:
      print("\n> " + document.metadata["source"] + ":")
      print(document.page_content)
      print(document.metadata["page"]+1)
  print("----------------------------------SOURCE DOCUMENTS---------------------------")


Enter a query: what are the patient's medical history?


> Question:
what are the patient's medical history?

> Answer:
Based on the medical report provided, here is what we can gather about the patient's medical history:
* The patient, Mr. Tan Ah Kow, is a 55-year-old male who has been seen by the same doctor, Dr. Tan Ah Moi, since November 2010.
* The patient has been experiencing a gradual deterioration in his cognitive ability and physical state over the years.
* The patient is currently unable to bathe or use the toilet on his own due to incontinence, but he is able to feed himself.
* The patient's son, Mr. Tan Ah Beng, accompanied him for the examination.
* The doctor was informed by Mr. Tan Ah Beng that the patient has been experiencing confusion, disorientation, and difficulty with memory.
* The patient underwent a mental state examination, during which he appeared to be in a normal mood and did not exhibit any psychotic symptoms.
* The doctor performed various tests and posed